In [46]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False
from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import requests
import numpy as np
import pandas as pd
import time

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'NanumGothicCoding'
plt.rcParams['font.size'] = 12

네이버는 100위까지 밖에 없어서 실시간, 일간, 주간, 월간 4가지가 있다 근데 장르가 안보이기에 클릭해서 페이지 이동 후 정보 스크롤 하는 식으로 해봄

In [2]:
# 일간 Top 100
targetSite = 'https://series.naver.com/novel/top100List.series' # 네이버 시리즈 Top 100 처음 띄우기
driver = webdriver.Chrome() 
driver.get(targetSite)
time.sleep(0.5)
request = requests.get(targetSite)
html = request.text
soup = BeautifulSoup(html, 'html.parser') # 크롤링 하려는 페이지 띄우기

NaverSeries_Day_Genre = [] # 장르 저장할 리스트
NaverSeries_Day_Titles = [] # 제목 저장할 리스트
NaverSeries_Day_Author = [] # 작가 저장할 리스트
NaverSeries_Day_Point = [] # 평점 리스트
NaverSeries_Day_Ranking = [i+1 for i in range(100) ]

# 첫 페이지 1~20위 클릭 후 정보 확인
#//*[@id="content"]/div/ul/li[1]/div[2]/h3/a # 1위
#//*[@id="content"]/div/ul/li[2]/div[2]/h3/a # 2위
#//*[@id="content"]/div/ul/li[3]/div[2]/h3/a # 3위

for i in range(1, 21, 1):
    driver.find_element(By.XPATH, f'//*[@id="content"]/div/ul/li[{i}]/div[2]/h3/a').click() # i번째 소설 들어감
    try:
        NaverSeries_Day_Titles.append(driver.find_element(By.XPATH, '//*[@id="content"]/div[1]/h2').text) # 제목
        NaverSeries_Day_Point.append(driver.find_element(By.XPATH, '//*[@id="content"]/div[1]/div[1]/em').text) # 평점
    except:
        NaverSeries_Day_Titles.append(driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/h2').text) # 제목
        NaverSeries_Day_Point.append(driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div[1]/em').text) # 평점        
    NaverSeries_Day_Genre.append(driver.find_element(By.XPATH, '//*[@id="content"]/ul[1]/li/ul/li[2]/span/a').text) # 장르
    NaverSeries_Day_Author.append(driver.find_element(By.XPATH, '//*[@id="content"]/ul[1]/li/ul/li[3]/a').text) # 작가
    driver.back()# i번 소설 크로링 후 전 페이지로
    
for i in range(1,5):
    # //*[@id="content"]/div/div[2]/a[1] # 21~40위 페이지 띄움
    # //*[@id="content"]/div/div[2]/a[2] # 41~60위 띄움
    # //*[@id="content"]/div/div[2]/a[3] # 61~80
    # //*[@id="content"]/div/div[2]/a[4] # 81~100
    driver.find_element(By.XPATH, f'//*[@id="content"]/div/div[2]/a[{i}]').click()
    for j in range(1, 21, 1):
        driver.find_element(By.XPATH, f'//*[@id="content"]/div/ul/li[{j}]/div[2]/h3/a').click() # i번째 소설 들어감
        try:
            NaverSeries_Day_Titles.append(driver.find_element(By.XPATH, '//*[@id="content"]/div[1]/h2').text) # 제목
            NaverSeries_Day_Point.append(driver.find_element(By.XPATH, '//*[@id="content"]/div[1]/div[1]/em').text) # 평점
        except:
            NaverSeries_Day_Titles.append(driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/h2').text) # 제목
            NaverSeries_Day_Point.append(driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div[1]/em').text) # 평점        
        NaverSeries_Day_Genre.append(driver.find_element(By.XPATH, '//*[@id="content"]/ul[1]/li/ul/li[2]/span/a').text) # 장르
        NaverSeries_Day_Author.append(driver.find_element(By.XPATH, '//*[@id="content"]/ul[1]/li/ul/li[3]/a').text) # 작가
        driver.back()# i번 소설 크로링 후 전 페이지로

In [3]:
NaverSeries_Day = pd.DataFrame(
    {'제목': NaverSeries_Day_Titles, '장르': NaverSeries_Day_Genre,'작가': NaverSeries_Day_Author, '평점': NaverSeries_Day_Point},
    index = NaverSeries_Day_Ranking) 

# 네이버 일간 top 100 끝

all_genres = pd.Series(NaverSeries_Day['장르'].unique())
#print(all_genres)
for genre in all_genres:
    NaverSeries_Day[genre] = (NaverSeries_Day['장르'] == genre)
NaverSeries_Day

,제목,장르,작가,평점,무협,현판,로판,판타지,로맨스
1,화산귀환 [독점],무협,비가,9.2,True,False,False,False,False
2,대기업 말단이 일을 잘함 [독점],현판,동면거북이,9.7,False,True,False,False,False
3,회귀자 생활백서(回歸者 生活白書),무협,넉울히,9.3,True,False,False,False,False
4,마교 교주의 부군이 되었다 [독점],무협,쥬레이,9.4,True,False,False,False,False
5,블레인필드의 상속자 [선공개],로판,파도저,9.4,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...
96,빼앗긴 내 아이 [독점],로맨스,하윤미,9.6,False,False,False,False,True
97,원수의 아기를 가졌다 [독점],로판,미니바라기,9.0,False,False,True,False,False
98,좌완으로 170km 던짐,현판,량주,7.6,False,True,False,False,False
99,축구부 방출 후 천재 투수가 되었다,현판,유선강,8.0,False,True,False,False,False


주간 top100도 위와 같은 식으로 실행 해봄

In [4]:
NaverSeries_Week_Genre = [] # 장르 저장할 리스트 생성 및 초기화
NaverSeries_Week_Titles = [] # 제목 저장할 리스트
NaverSeries_Week_Author = [] # 작가 저장할 리스트
NaverSeries_Week_Point = [] # 평점 리스트
NaverSeries_Week_Ranking = [i+1 for i in range(100) ]

driver.find_element(By.XPATH, '//*[@id="container"]/div[1]/ul/li[3]/a').click()

for i in range(1, 21, 1):
    driver.find_element(By.XPATH, f'//*[@id="content"]/div/ul/li[{i}]/div[2]/h3/a').click() # i번째 소설 들어감
    try:
        NaverSeries_Week_Titles.append(driver.find_element(By.XPATH, '//*[@id="content"]/div[1]/h2').text) # 제목
        NaverSeries_Week_Point.append(driver.find_element(By.XPATH, '//*[@id="content"]/div[1]/div[1]/em').text) # 평점
    except:
        NaverSeries_Week_Titles.append(driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/h2').text) # 제목
        NaverSeries_Week_Point.append(driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div[1]/em').text) # 평점        
    NaverSeries_Week_Genre.append(driver.find_element(By.XPATH, '//*[@id="content"]/ul[1]/li/ul/li[2]/span/a').text) # 장르
    NaverSeries_Week_Author.append(driver.find_element(By.XPATH, '//*[@id="content"]/ul[1]/li/ul/li[3]/a').text) # 작가
    driver.back()# i번 소설 크로링 후 전 페이지로
    
for i in range(1,5):
    # //*[@id="content"]/div/div[2]/a[1] # 21~40위 페이지 띄움
    # //*[@id="content"]/div/div[2]/a[2] # 41~60위 띄움
    # //*[@id="content"]/div/div[2]/a[3] # 61~80
    # //*[@id="content"]/div/div[2]/a[4] # 81~100
    driver.find_element(By.XPATH, f'//*[@id="content"]/div/div[2]/a[{i}]').click()
    for j in range(1, 21, 1):
        driver.find_element(By.XPATH, f'//*[@id="content"]/div/ul/li[{j}]/div[2]/h3/a').click() # i번째 소설 들어감
        try:
            NaverSeries_Week_Titles.append(driver.find_element(By.XPATH, '//*[@id="content"]/div[1]/h2').text) # 제목
            NaverSeries_Week_Point.append(driver.find_element(By.XPATH, '//*[@id="content"]/div[1]/div[1]/em').text) # 평점
        except:
            NaverSeries_Week_Titles.append(driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/h2').text) # 제목
            NaverSeries_Week_Point.append(driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div[1]/em').text) # 평점        
        NaverSeries_Week_Genre.append(driver.find_element(By.XPATH, '//*[@id="content"]/ul[1]/li/ul/li[2]/span/a').text) # 장르
        NaverSeries_Week_Author.append(driver.find_element(By.XPATH, '//*[@id="content"]/ul[1]/li/ul/li[3]/a').text) # 작가
        driver.back()# i번 소설 크로링 후 전 페이지로
        

In [5]:
NaverSeries_Week = pd.DataFrame(
    {'제목': NaverSeries_Week_Titles, '장르': NaverSeries_Week_Genre,'작가': NaverSeries_Week_Author, '평점': NaverSeries_Week_Point},
    index = NaverSeries_Week_Ranking) 

# 네이버 일간 top 100 끝

all_genres = pd.Series(NaverSeries_Week['장르'].unique())
#print(all_genres)
for genre in all_genres:
    NaverSeries_Week[genre] = (NaverSeries_Week['장르'] == genre)
NaverSeries_Week

,제목,장르,작가,평점,무협,판타지,로판,현판,로맨스
1,화산귀환 [독점],무협,비가,9.2,True,False,False,False,False
2,오늘만 사는 기사 [독점],판타지,소울풍,9.8,False,True,False,False,False
3,당신을 배신한 나를 구원해 줄래요 [선공개],로판,바애,8.8,False,False,True,False,False
4,블레인필드의 상속자 [선공개],로판,파도저,9.4,False,False,True,False,False
5,2레벨로 회귀한 무신,판타지,염비,9.3,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...
96,각자의 본능대로 [독점],로맨스,안테,9.8,False,False,False,False,True
97,무공 쓰는 외과 의사,현판,윤백현,9.5,False,False,False,True,False
98,약먹는 천재마법사,판타지,글근육,9.9,False,True,False,False,False
99,네가 길들인 남자 [독점],로맨스,레이먼,9.6,False,False,False,False,True


In [6]:
NaverSeries_Month_Genre = [] # 장르 저장할 리스트 생성 및 초기화
NaverSeries_Month_Titles = [] # 제목 저장할 리스트
NaverSeries_Month_Author = [] # 작가 저장할 리스트
NaverSeries_Month_Point = [] # 평점 리스트
NaverSeries_Month_Ranking = [i+1 for i in range(100) ]

driver.find_element(By.XPATH, '//*[@id="container"]/div[1]/ul/li[4]/a').click()

for i in range(1, 21, 1):
    
    if driver.find_element(By.XPATH, f'//*[@id="content"]/div/ul/li[{i}]/div[2]/h3/a').text == '19금': # 19금 소설일 때
        pass
    NaverSeries_Now_Ranking.append(driver.find_element(By.XPATH, '//*[@id="content"]/div/ul/li[1]/div[1]/span[1]/em').text) # 순위
    driver.find_element(By.XPATH, f'//*[@id="content"]/div/ul/li[{i}]/div[2]/h3/a').click() # i번째 소설 들어감
    
    try:
        NaverSeries_Month_Titles.append(driver.find_element(By.XPATH, '//*[@id="content"]/div[1]/h2').text) # 제목
        NaverSeries_Month_Point.append(driver.find_element(By.XPATH, '//*[@id="content"]/div[1]/div[1]/em').text) # 평점
    except:
        NaverSeries_Month_Titles.append(driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/h2').text) # 제목
        NaverSeries_Month_Point.append(driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div[1]/em').text) # 평점        
    NaverSeries_Month_Genre.append(driver.find_element(By.XPATH, '//*[@id="content"]/ul[1]/li/ul/li[2]/span/a').text) # 장르
    NaverSeries_Month_Author.append(driver.find_element(By.XPATH, '//*[@id="content"]/ul[1]/li/ul/li[3]/a').text) # 작가
    driver.back()# i번 소설 크로링 후 전 페이지로
    
for i in range(1,5):
    # //*[@id="content"]/div/div[2]/a[1] # 21~40위 페이지 띄움
    # //*[@id="content"]/div/div[2]/a[2] # 41~60위 띄움
    # //*[@id="content"]/div/div[2]/a[3] # 61~80
    # //*[@id="content"]/div/div[2]/a[4] # 81~100
    driver.find_element(By.XPATH, f'//*[@id="content"]/div/div[2]/a[{i}]').click()
    for j in range(1, 21, 1):
        driver.find_element(By.XPATH, f'//*[@id="content"]/div/ul/li[{j}]/div[2]/h3/a').click() # i번째 소설 들어감
        try:
            NaverSeries_Month_Titles.append(driver.find_element(By.XPATH, '//*[@id="content"]/div[1]/h2').text) # 제목
            NaverSeries_Month_Point.append(driver.find_element(By.XPATH, '//*[@id="content"]/div[1]/div[1]/em').text) # 평점
        except:
            NaverSeries_Month_Titles.append(driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/h2').text) # 제목
            NaverSeries_Month_Point.append(driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div[1]/em').text) # 평점        
        NaverSeries_Month_Genre.append(driver.find_element(By.XPATH, '//*[@id="content"]/ul[1]/li/ul/li[2]/span/a').text) # 장르
        NaverSeries_Month_Author.append(driver.find_element(By.XPATH, '//*[@id="content"]/ul[1]/li/ul/li[3]/a').text) # 작가
        driver.back()# i번 소설 크로링 후 전 페이지로

In [7]:
NaverSeries_Month = pd.DataFrame(
    {'제목': NaverSeries_Month_Titles, '장르': NaverSeries_Month_Genre,'작가': NaverSeries_Month_Author, '평점': NaverSeries_Month_Point},
    index = NaverSeries_Month_Ranking) 

# 네이버 일간 top 100 끝

all_genres = pd.Series(NaverSeries_Month['장르'].unique())
#print(all_genres)
for genre in all_genres:
    NaverSeries_Month[genre] = (NaverSeries_Month['장르'] == genre)
NaverSeries_Month

,제목,장르,작가,평점,무협,판타지,현판,로맨스,로판
1,화산귀환 [독점],무협,비가,9.2,True,False,False,False,False
2,절대회귀 [독점],무협,장영훈,9.8,True,False,False,False,False
3,오늘만 사는 기사 [독점],판타지,소울풍,9.8,False,True,False,False,False
4,괴력 난신(怪力 亂神),무협,한중월야,9.7,True,False,False,False,False
5,회귀수선전(回歸修仙傳),무협,엄청난,9.6,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...
96,유혹은 교만하게 [선공개],로맨스,문유하,9.7,False,False,False,True,False
97,악당 가족이 독립을 반대한다 [독점],로판,이흰,9.9,False,False,False,False,True
98,검술 명가의 대마법사 막내딸 [독점],로판,윤하월,9.8,False,False,False,False,True
99,훔친 용이지만 반품될까요 [선공개],로판,Onmi,8.6,False,False,False,False,True


In [53]:
NaverSeries_Now_Genre = [] # 장르 저장할 리스트 생성 및 초기화
NaverSeries_Now_Titles = [] # 제목 저장할 리스트
NaverSeries_Now_Author = [] # 작가 저장할 리스트
NaverSeries_Now_Point = [] # 평점 리스트
NaverSeries_Now_Ranking = [] # 랭킹

driver.find_element(By.XPATH, '//*[@id="container"]/div[1]/ul/li[1]/a').click()
# //*[@id="content"]/div/ul/li[9]/div[2]/h3/em # 19금 걸려서 로그인창으로 이동됨

for i in range(1, 21, 1): # 첫 페이지 크롤링 
    print(i)
    NaverSeries_Now_Ranking.append(driver.find_element(By.XPATH, '//*[@id="content"]/div/ul/li[1]/div[1]/span[1]/em').text) # 순위 리스트
    
    try:
        head = driver.find_element(By.XPATH, f'//*[@id="content"]/div/ul/li[{i}]/div[2]/h3/em').text
        # head = 19금 or new or up or ''
        if head == '19금':
            i+=1
            if i > 20:
                break
            driver.find_element(By.XPATH, f'//*[@id="content"]/div/ul/li[{i+1}]/div[2]/h3/a').click() # i+1번째 소설 들어감
            i-=1
        else:    
            driver.find_element(By.XPATH, f'//*[@id="content"]/div/ul/li[{i}]/div[2]/h3/a').click() # i번째 소설 들어감
            
        try:
            title=driver.find_element(By.XPATH, '//*[@id="content"]/div[1]/h2').text
            point=driver.find_element(By.XPATH, '//*[@id="content"]/div[1]/div[1]/em').text
            NaverSeries_Now_Titles.append(title) # 제목
            NaverSeries_Now_Point.append(point) # 평점
        except:
            title=driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/h2').text
            point=driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div[1]/em').text
            NaverSeries_Now_Titles.append(title) # 제목
            NaverSeries_Now_Point.append(point) # 평점
        genre = driver.find_element(By.XPATH, '//*[@id="content"]/ul[1]/li/ul/li[2]/span/a').text # 장르
        author = driver.find_element(By.XPATH, '//*[@id="content"]/ul[1]/li/ul/li[3]/a').text # 작가
        NaverSeries_Now_Genre.append(genre) # 장르
        NaverSeries_Now_Author.append(author) # 작가 
        driver.back() # 전 페이지로
    except NoSuchElementException: # head가 없는 경우
        driver.find_element(By.XPATH, f'//*[@id="content"]/div/ul/li[{i}]/div[2]/h3/a').click() # i번째 소설 들어감
        try:
            title=driver.find_element(By.XPATH, '//*[@id="content"]/div[1]/h2').text
            point=driver.find_element(By.XPATH, '//*[@id="content"]/div[1]/div[1]/em').text
            NaverSeries_Now_Titles.append(title) # 제목
            NaverSeries_Now_Point.append(point) # 평점
        except:
            title=driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/h2').text
            point=driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div[1]/em').text
            NaverSeries_Now_Titles.append(title) # 제목
            NaverSeries_Now_Point.append(point) # 평점
        genre = driver.find_element(By.XPATH, '//*[@id="content"]/ul[1]/li/ul/li[2]/span/a').text # 장르
        author = driver.find_element(By.XPATH, '//*[@id="content"]/ul[1]/li/ul/li[3]/a').text # 작가
        NaverSeries_Now_Genre.append(genre) # 장르
        NaverSeries_Now_Author.append(author) # 작가 
        driver.back() # 전 페이지로
    print(f'{i} {head} 제목:{title}, 작가:{author}, 장르{genre}, 평점{point}')
'''    
for i in range(1,5):
    # //*[@id="content"]/div/div[2]/a[1] # 21~40위 페이지 띄움
    # //*[@id="content"]/div/div[2]/a[2] # 41~60위 띄움
    # //*[@id="content"]/div/div[2]/a[3] # 61~80
    # //*[@id="content"]/div/div[2]/a[4] # 81~100
    driver.find_element(By.XPATH, f'//*[@id="content"]/div/div[2]/a[{i}]').click()
    for j in range(1, 21, 1):
        NaverSeries_Now_Ranking.append(driver.find_element(By.XPATH, '//*[@id="content"]/div/ul/li[1]/div[1]/span[1]/em').text) # 순위
        
        try:
            text19 = driver.find_element(By.XPATH, f'//*[@id="content"]/div/ul/li[{i}]/div[2]/h3/em').text
        except NoSuchElementException:
            print("요소를 찾지 못했습니다.")

        if text19 == '19금':
            print('19금 소설 임')
            i+=1
        
        driver.find_element(By.XPATH, f'//*[@id="content"]/div/ul/li[{j}]/div[2]/h3/a').click() # j번째 소설 들어감
        try:
            NaverSeries_Now_Titles.append(driver.find_element(By.XPATH, '//*[@id="content"]/div[1]/h2').text) # 제목
            NaverSeries_Now_Point.append(driver.find_element(By.XPATH, '//*[@id="content"]/div[1]/div[1]/em').text) # 평점
        except:
            NaverSeries_Now_Titles.append(driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/h2').text) # 제목
            NaverSeries_Now_Point.append(driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div[1]/em').text) # 평점        
        NaverSeries_Now_Genre.append(driver.find_element(By.XPATH, '//*[@id="content"]/ul[1]/li/ul/li[2]/span/a').text) # 장르
        NaverSeries_Now_Author.append(driver.find_element(By.XPATH, '//*[@id="content"]/ul[1]/li/ul/li[3]/a').text) # 작가
        if text19 == '19금':
            i-=1
        driver.back()# i번 소설 크로링 후 전 페이지로
'''        

1
1 신규 제목:나의 조선에 유교는 필요없다, 작가:정동갑제, 장르판타지, 평점9.0
2
2 신규 제목:1500억을 받았다, 작가:먼지운, 장르현판, 평점9.3
3
3 신규 제목:오버로드 [단행본], 작가:마루야마 쿠가네, 장르라이트노벨, 평점9.3
4
4 새로운 에피소드 제목:회귀자 생활백서(回歸者 生活白書), 작가:넉울히, 장르무협, 평점9.3
5
5 새로운 에피소드 제목:마교 교주의 부군이 되었다 [독점], 작가:쥬레이, 장르무협, 평점9.4
6
6 새로운 에피소드 제목:화산귀환 [독점], 작가:비가, 장르무협, 평점9.2
7
7 새로운 에피소드 제목:대기업 말단이 일을 잘함 [독점], 작가:동면거북이, 장르현판, 평점9.7
8
8 19금 제목:위저드 스톤 [단행본], 작가:이경훈, 장르판타지, 평점8.2
9
9 19금 제목:십자성-전왕의 검 [단행본], 작가:허담, 장르무협, 평점8.0
10
10 19금 제목:위저드 스톤 [단행본], 작가:이경훈, 장르판타지, 평점8.2
11
11 19금 제목:원수의 아기를 가졌다 [독점], 작가:미니바라기, 장르로판, 평점9.0
12
12 19금 제목:최종 보스 : 빛을 향해 달리는 그림자 [단행본], 작가:무장, 장르현판, 평점10.0
13
13 19금 제목:군림천하 [단행본], 작가:용대운, 장르무협, 평점8.8
14
14 신규 제목:당신의 머리 위에 [단행본], 작가:박건, 장르판타지, 평점0.0
15
15 신규 제목:소드엠페러 [단행본], 작가:김정률, 장르판타지, 평점7.8
16
16 19금 제목:사천당가의 검신급 소가주가 되었다, 작가:화계火鷄, 장르무협, 평점7.6
17
17 신규 제목:회귀한 재벌은 미국을 사기로했다, 작가:연태량, 장르현판, 평점8.7
18
18 새로운 에피소드 제목:사천당가의 검신급 소가주가 되었다, 작가:화계火鷄, 장르무협, 평점7.6
19
20 19금 제목:근본있는 소드마스터, 작가:로튼애플, 장르판타지, 평점8.9
20
20 신규 제목:근본있는 소드마스터, 작가:로튼애플, 장르

'    \nfor i in range(1,5):\n    # //*[@id="content"]/div/div[2]/a[1] # 21~40위 페이지 띄움\n    # //*[@id="content"]/div/div[2]/a[2] # 41~60위 띄움\n    # //*[@id="content"]/div/div[2]/a[3] # 61~80\n    # //*[@id="content"]/div/div[2]/a[4] # 81~100\n    driver.find_element(By.XPATH, f\'//*[@id="content"]/div/div[2]/a[{i}]\').click()\n    for j in range(1, 21, 1):\n        NaverSeries_Now_Ranking.append(driver.find_element(By.XPATH, \'//*[@id="content"]/div/ul/li[1]/div[1]/span[1]/em\').text) # 순위\n        \n        try:\n            text19 = driver.find_element(By.XPATH, f\'//*[@id="content"]/div/ul/li[{i}]/div[2]/h3/em\').text\n        except NoSuchElementException:\n            print("요소를 찾지 못했습니다.")\n\n        if text19 == \'19금\':\n            print(\'19금 소설 임\')\n            i+=1\n        \n        driver.find_element(By.XPATH, f\'//*[@id="content"]/div/ul/li[{j}]/div[2]/h3/a\').click() # j번째 소설 들어감\n        try:\n            NaverSeries_Now_Titles.append(driver.find_element(By.XPATH, \'/

In [50]:
print(len(NaverSeries_Now_Titles), len(NaverSeries_Now_Genre), len(NaverSeries_Now_Author), len(NaverSeries_Now_Point))

12 12 12 12


In [ ]:
NaverSeries_Now = pd.DataFrame(
    {'제목': NaverSeries_Now_Titles, '장르': NaverSeries_Now_Genre,'작가': NaverSeries_Now_Author, '평점': NaverSeries_Now_Point},
    index = NaverSeries_Now_Ranking) 

# 네이버 일간 top 100 끝

all_genres = pd.Series(NaverSeries_Now['장르'].unique())
#print(all_genres)
for genre in all_genres:
    NaverSeries_Now[genre] = (NaverSeries_Now['장르'] == genre)
NaverSeries_Now

In [ ]:
NaverSeries_Now.to_csv('./data/NaverSeries_Now100.txt', sep = '\t')
NaverSeries_Day.to_csv('./data/NaverSeries_Day100.txt', sep = '\t')
NaverSeries_Week.to_csv('./data/NaverSeries_Week100.txt', sep = '\t')
NaverSeries_Month.to_csv('./data/NaverSeries_Month100.txt', sep = '\t')

In [27]:
i=5
print(i) # 5
i+1
print(i)
i+=1
print(i)

5
5
6
